In [1]:
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
#from sklearn.metrics import classification_report, confusion_matrix

#Start


In [2]:
train_data_path = 'D:/paddy_leaf/Rice Leaf Disease Images2-small/train'
test_data_path = 'D:/paddy_leaf/Rice Leaf Disease Images2-small/test'
img_rows = 150
img_cols = 150
epochs = 60
batch_size = 4
num_of_train_samples = 212
num_of_test_samples = 60

#Image Generator
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_path,
                                                    target_size=(img_rows, img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')
    
validation_generator = test_datagen.flow_from_directory(test_data_path,
                                                        target_size=(img_rows, img_cols),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')


#Train



Found 2164 images belonging to 6 classes.
Found 1128 images belonging to 6 classes.


In [3]:
# Build model
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_rows, img_cols, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(128, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [4]:
model.fit_generator(train_generator,
                    steps_per_epoch=num_of_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=num_of_test_samples // batch_size)

D:\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
53/53 [==============================] - 19s 318ms/step - loss: 1.8825 - accuracy: 0.2453 - val_loss: 1.4604 - val_accuracy: 0.5000
Epoch 2/60
53/53 [==============================] - 19s 350ms/step - loss: 1.5285 - accuracy: 0.4292 - val_loss: 1.2824 - val_accuracy: 0.5833ccuracy
Epoch 3/60
53/53 [==============================] - 20s 376ms/step - loss: 1.2891 - accuracy: 0.4528 - val_loss: 1.2104 - val_accuracy: 0.5333
Epoch 4/60
53/53 [==============================] - 16s 302ms/step - loss: 1.2698 - accuracy: 0.4528 - val_loss: 1.0565 - val_accuracy: 0.6833
Epoch 5/60
53/53 [==============================] - 18s 347ms/step - loss: 1.2389 - accuracy: 0.5425 - val_loss: 1.1549 - val_accuracy: 0.6167
Epoch 6/60
53/53 [==============================] - 18s 350ms/step - loss: 1.3095 - accuracy: 0.4575 - val_loss: 0.9445 - val_accuracy: 0.7333
Epoch 7/60
53/53 [==============================] - 17s 317ms/step - loss: 0.9725 - accuracy: 0.6132 - val_loss: 0.9146 - val_accuracy:

In [5]:
 #Confution Matrix
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
    
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    
    """ This function prints and plots the confusion matrix.
       Normalization can be applied by setting `normalize=True`."""
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
   
    print(cm)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
     plt.text(j, i, cm[i, j],
              horizontalalignment="center",
              color="white" if cm[i, j] > thresh else "black")
     plt.tight_layout()
     plt.ylabel('True label')
     plt.xlabel('Predicted label')

In [6]:
Y_pred = model.predict_generator( validation_generator, 15 )
y_pred = np.argmax(Y_pred, axis=1)

D:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [7]:
cm=confusion_matrix(validation_generator.classes, y_pred)
plot_confusion_matrix(cm,['Bacterialblight','Blast','Brownspot','Healthy','Leaf smut','Tungro'])# use your class labels"

ValueError: Found input variables with inconsistent numbers of samples: [1128, 60]